In [57]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from sklearn.model_selection import cross_val_score

stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()

classificacoes = pd.read_csv('emails.csv')
textosPuros = classificacoes['email']
frases = textosPuros.str.lower()
textosQuebrados = [nltk.tokenize.word_tokenize(frase) for frase in frases]

dicionario = set()

for lista in textosQuebrados:
    validas = [stemmer.stem(palavra) for palavra in lista if palavra not in stopwords and len(palavra) > 2]
    dicionario.update(validas)
    
total_palavras = len(dicionario)
tuplas = zip(dicionario, range(total_palavras))
tradutor = {palavra:indice for palavra, indice in tuplas}
    
vetores_texto = [vetorizar_texto(texto, tradutor, stemmer) for texto in textosQuebrados]

marcas = classificacoes['classificacao']

X = vetores_texto
Y = marcas

porcentagem_treino = 0.8

tamanho_treino = int(porcentagem_treino * len(Y))
tamanho_validacao = len(Y) - tamanho_treino

treino_dados = X[:tamanho_treino]
treino_marcacoes = Y[:tamanho_treino]

validacao_dados = X[tamanho_treino:]
validacao_marcacoes = Y[tamanho_treino:]

In [47]:
def vetorizar_texto(texto, tradutor, stemmer):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if len(palavra) > 0:
            raiz = stemmer.stem(palavra)
            if raiz in tradutor:
                posicao = tradutor[raiz]
                vetor[posicao] += 1
    
    return vetor

In [43]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_acerto = np.mean(scores)
    msg = 'Taxa de acerto do {0}: {1}'.format(nome, taxa_acerto)
    print(msg)
    return taxa_acerto

In [44]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)
    
    acertos = resultado == validacao_marcacoes
    
    total_acertos = sum(acertos)
    total_elementos = len(validacao_marcacoes)
    
    taxa_acerto = 100.0 * total_acertos / total_elementos
    
    msg = "Taxa de acerto do vencedor entre os algoritmos no mundo real: {0}".format(taxa_acerto)
    
    print(msg)

In [58]:
resultados = {}

# treinando com OneVsRest
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, tol = 1))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

# treinando com OneVsOne
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0, tol = 1))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

# treinando com MultinomialNB
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

# treinando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier(random_state = 0)
resultadoAdaBoost = fit_and_predict("AdaBoost", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

maximo = max(resultados)
vencedor = resultados[maximo]

vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_acerto_base)

total_elementos = len(validacao_dados)
print("Total de teste: %d" % total_elementos)

Taxa de acerto do OneVsRest: 0.775
Taxa de acerto do OneVsOne: 0.75
Taxa de acerto do MultinomialNB: 0.8
Taxa de acerto do AdaBoost: 0.5833333333333333
Taxa de acerto do vencedor entre os algoritmos no mundo real: 77.77777777777777
Taxa de acerto base: 44.444444
Total de teste: 9
